In [17]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

In [71]:
headers_orig = open('headers_orig.txt', 'r')
headers_orig = headers_orig.read().strip().split('\n')
orig = pd.read_csv("sample_orig_2018.txt", index_col=False, names=headers_orig, sep="|")

headers_svcg = open('headers_svcg.txt', 'r')
headers_svcg = headers_svcg.read().strip().split('\n')
svcg = pd.read_csv("sample_svcg_2018.txt", index_col=False, names=headers_svcg, sep="|")

new_svcg = svcg.head(50000)
new_svcg = new_svcg.join(orig.set_index("Loan Sequence Number"), on="Loan Sequence Number", how="left")
new_svcg.to_csv('new_sample.csv', index=False) 

svcg = new_svcg

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [93]:
svcg = new_svcg

def current_loan_helper(val):
    try:
        if int(val) >= 3:
            return 1
        else:
            return 0
    except:
        return 1
    
def zero_balance_helper(val):
    target = [3,6,9]
    if val in target:
        return 1
    else:
        return 0
    
def default_helper(val):
    if val > 0:
        return 1
    else:
        return 0
    
svcg['Zero Balance Code'] = svcg['Zero Balance Code'].apply(zero_balance_helper)
svcg['Current Loan Delinquency Status'] = svcg['Current Loan Delinquency Status'].apply(current_loan_helper)
svcg = svcg.assign(**{'Default': svcg['Current Loan Delinquency Status'] + svcg['Zero Balance Code']})
svcg['Default'] = svcg['Default'].apply(default_helper)
cols = orig.columns.to_list()
cols.append("Default")
svcg = svcg.filter(items=cols)
svcg = svcg.drop_duplicates()
svcg

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Default
0,784,201803,9,204802,49180.0,0,1,I,75,39,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
22,693,201803,N,203302,24340.0,0,1,P,80,41,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N,0
49,731,201803,9,204802,NaN,0,1,P,45,37,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
75,763,201802,Y,204801,NaN,30,1,P,95,34,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
110,755,201803,9,204802,49700.0,0,1,P,80,33,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49885,787,201803,9,204802,10900.0,12,1,P,85,42,...,1,"WELLS FARGO BANK, N.A.","WELLS FARGO BANK, N.A.",NaN,NaN,9,NaN,2,N,0
49920,713,201803,Y,204802,29404.0,0,1,P,80,44,...,2,QUICKEN LOANS INC.,NEW RESIDENTIAL MORTGAGE LLC,NaN,NaN,9,NaN,2,N,0
49955,735,201803,Y,204802,33124.0,30,1,P,95,50,...,2,QUICKEN LOANS INC.,NEW RESIDENTIAL MORTGAGE LLC,NaN,NaN,9,NaN,2,N,0
49990,792,201803,Y,204802,35614.0,12,1,P,85,34,...,1,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,H,NaN,2,N,0


In [94]:
simpleImputer = SimpleImputer(missing_values='99', strategy='constant', fill_value='SF')
svcg['Property Type'] = simpleImputer.fit_transform(svcg[['Property Type']])

simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='P')
svcg['Loan Purpose'] = simpleImputer.fit_transform(svcg[['Loan Purpose']])

simpleImputer = SimpleImputer(missing_values=9, strategy='constant', fill_value=1)
svcg['Number of Borrowers'] = simpleImputer.fit_transform(svcg[['Number of Borrowers']])

simpleImputer = SimpleImputer(missing_values=9, strategy='constant', fill_value=1)
svcg['Number of Borrowers'] = simpleImputer.fit_transform(svcg[['Number of Borrowers']])

In [95]:
simpleImputer = SimpleImputer(strategy='constant', fill_value='N')
svcg['Super Conforming Flag'] = simpleImputer.fit_transform(svcg[['Super Conforming Flag']])
oneHot = OneHotEncoder(sparse=False)
svcg['Super Conforming Flag']  = pd.DataFrame(oneHot.fit_transform(svcg[['Super Conforming Flag']]))[0].values

oneHot = OneHotEncoder(sparse=False)
svcg['Program Indicator']  = pd.DataFrame(oneHot.fit_transform(svcg[['Program Indicator']]))[0].values

oneHot = OneHotEncoder(sparse=False)
svcg['Interest Only (I/O) Indicator']  = pd.DataFrame(oneHot.fit_transform(svcg[['Interest Only (I/O) Indicator']]))[0].values

In [96]:
svcg = svcg.drop('Property State', axis=1)
svcg = svcg.drop('Postal Code', axis=1)
svcg = svcg.drop('Loan Sequence Number', axis=1)
svcg = svcg.drop('Seller Name', axis=1)
svcg = svcg.drop('Servicer Name', axis=1)
svcg = svcg.drop('Pre-HARP Loan Sequence Number', axis=1)
svcg = svcg[svcg['HARP Indicator'] != 'Y']
svcg = svcg.drop('HARP Indicator', axis=1)
svcg = svcg.drop('Property Valuation Method', axis=1)
svcg

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Prepayment Penalty Mortgage (PPM) Flag,Amortization Type (Formerly Product Type),Property Type,Loan Purpose,Original Loan Term,Number of Borrowers,Super Conforming Flag,Program Indicator,Interest Only (I/O) Indicator,Default
0,784,201803,9,204802,49180.0,0,1,I,75,39,...,N,FRM,PU,C,360,1,1.0,1.0,1.0,0
22,693,201803,N,203302,24340.0,0,1,P,80,41,...,N,FRM,SF,P,180,2,1.0,1.0,1.0,0
49,731,201803,9,204802,NaN,0,1,P,45,37,...,N,FRM,SF,C,360,1,1.0,1.0,1.0,0
75,763,201802,Y,204801,NaN,30,1,P,95,34,...,N,FRM,SF,P,360,1,1.0,1.0,1.0,0
110,755,201803,9,204802,49700.0,0,1,P,80,33,...,N,FRM,PU,C,360,2,1.0,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49885,787,201803,9,204802,10900.0,12,1,P,85,42,...,N,FRM,SF,C,360,1,1.0,1.0,1.0,0
49920,713,201803,Y,204802,29404.0,0,1,P,80,44,...,N,FRM,SF,P,360,2,1.0,1.0,1.0,0
49955,735,201803,Y,204802,33124.0,30,1,P,95,50,...,N,FRM,SF,P,360,2,1.0,1.0,1.0,0
49990,792,201803,Y,204802,35614.0,12,1,P,85,34,...,N,FRM,CO,P,360,1,1.0,0.0,1.0,0
